<a href="https://colab.research.google.com/github/ArpitaChatterjee/AmericanSignLanguage/blob/main/ASL_Integrate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale = 1./255, 
                             shear_range = 0.2,
                             validation_split = 0.1, 
                             zoom_range = 0.2,  
                             horizontal_flip = True,
                             samplewise_center = True, 
                             samplewise_std_normalization = True)

training_set = datagen.flow_from_directory('/content/drive/My Drive/Colab Notebooks/ASL/ASL-DATASETS/asl_alphabet_train', 
                                           target_size = (64, 64), 
                                           batch_size = 16,
                                           class_mode = 'categorical',
                                           subset = 'training')

test_set = datagen.flow_from_directory('/content/drive/My Drive/Colab Notebooks/ASL/ASL-DATASETS/asl_alphabet_train',
                                       target_size = (64, 64), 
                                       batch_size = 16,
                                       class_mode = 'categorical', 
                                       subset = 'validation')

Found 57319 images belonging to 29 classes.
Found 6367 images belonging to 29 classes.


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Dropout, BatchNormalization
# Initialising the CNN
cnn = Sequential()

#First Layer
cnn.add(Conv2D(filters = 64, kernel_size = (4, 4), 
                      input_shape = (64, 64, 3), activation = 'relu'))

cnn.add(Conv2D(filters = 64, kernel_size = (4, 4), strides = 2,  activation = 'relu'))

cnn.add(Dropout(0.5))

cnn.add(BatchNormalization(axis = 3, momentum = 0.8))

#Second Layer
cnn.add(Conv2D(filters = 128, kernel_size = (4, 4), activation = 'relu'))

cnn.add(Conv2D(filters = 128, kernel_size = (4, 4), strides = 2,  activation = 'relu'))

cnn.add(Dropout(0.5))

cnn.add(BatchNormalization(axis = 3, momentum = 0.8))

#Third Layer

cnn.add(Conv2D(filters = 256, kernel_size = (4, 4), activation = 'relu'))

cnn.add(Conv2D(filters = 256, kernel_size = (4, 4), strides = 2,  activation = 'relu'))

cnn.add(Dropout(0.5))

cnn.add(BatchNormalization(axis = 3, momentum = 0.8))

# Flattening
cnn.add(Flatten())

cnn.add(Dropout(0.5))

# Hidden Layer and Output Layer
cnn.add(Dense(units = 512, activation = 'relu'))
cnn.add(Dense(units = 29, activation = 'softmax'))

#Compiling the CNN
cnn.compile(optimizer= 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
r = cnn.fit_generator(training_set,                  
                  epochs = 10,
                  validation_data = test_set,
                  steps_per_epoch = len(training_set), 
                  validation_steps = len(test_set))

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
 211/3583 [>.............................] - ETA: 6:22:35 - loss: 3.8114 - accuracy: 0.0847

In [ ]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = cnn.evaluate(test_set, batch_size=16)
print("test loss, test acc:", results)